In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt 
import copy
import timeit
import time
import pickle
import subprocess
import os
import re


In [ ]:
import os
def plotp (m):
    data=m[::-1,:]
    heatmap = plt.pcolor(data, cmap=plt.cm.tab20, edgecolors='grey', linewidths=2)
    for y in range(data.shape[0]):
        for x in range(data.shape[1]):
            plt.text(x + 0.5, y + 0.5, '%.0f' % data[y, x],
                     horizontalalignment='center',
                     verticalalignment='center',
                     )
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.show()
def start(w,h):
    arr = np.zeros(w*h,dtype=int) 
    arr[:random.randrange(0, w*h+1)]  = 1
    np.random.shuffle(arr)
    g = np.reshape(arr,(w,h))
    return g
g=start(5,5)

In [ ]:
import numpy as np

def balancednessDetection(field):
    field = np.array(field)  # Ensure field is a numpy array for efficient operations

    def rectangleRecognition(field):
        rows, cols = field.shape
        for i in range(rows - 1):
            for j in range(cols - 1):
                if field[i, j] > 0:
                    if field[i + 1, j] > 0 and field[i, j + 1] > 0 and field[i + 1, j + 1] > 0:
                        if (i + 2 < rows and field[i + 2, j] > 0 and field[i + 2, j + 1] > 0) or \
                           (j + 2 < cols and field[i, j + 2] > 0 and field[i + 1, j + 2] > 0):
                            return True
        return False

    def squaresRecognition(field):
        rows, cols = field.shape
        for i in range(1, rows - 1):
            for j in range(1, cols - 1):
                if field[i, j] == 1 and \
                   field[i, j - 1] == 1 and field[i - 1, j] == 1 and \
                   field[i + 1, j] == 1 and field[i, j + 1] == 1:
                    if (field[i - 1, j + 1] == 1 and field[i + 1, j - 1] == 1) or \
                       (field[i - 1, j - 1] == 1 and field[i + 1, j + 1] == 1):
                        return True
        return False

    def cycles(field):
        pixels = set(zip(*np.where(field == 1)))
        rows, cols = field.shape

        def get_neighbors(x, y):
            neighbors = []
            if x > 0 and field[x - 1, y] == 1:
                neighbors.append((x - 1, y))
            if x < rows - 1 and field[x + 1, y] == 1:
                neighbors.append((x + 1, y))
            if y > 0 and field[x, y - 1] == 1:
                neighbors.append((x, y - 1))
            if y < cols - 1 and field[x, y + 1] == 1:
                neighbors.append((x, y + 1))
            return neighbors

        while pixels:
            root = pixels.pop()
            stack = [root]
            pred = {root: root}
            used = {root: set()}

            while stack:
                z = stack.pop()
                zused = used[z]
                i, j = z
                neighbors = get_neighbors(i, j)

                for nb in neighbors:
                    if nb not in used:
                        pred[nb] = z
                        stack.append(nb)
                        used[nb] = {z}
                    elif nb == z:
                        continue
                    elif nb not in zused:
                        pn = used[nb]
                        cycle = [nb, z]
                        p = pred[z]
                        while p not in pn:
                            cycle.append(p)
                            p = pred[p]
                        cycle.append(p)
                        if len(cycle) > 4:
                            return True
                        used[nb].add(z)
            pixels -= set(pred)
        return False

    if rectangleRecognition(field) or squaresRecognition(field) or cycles(field):
        return False
    else:
        return True

# Example usage
g = [
    [1, 1, 0, 0],
    [1, 1, 1, 0],
    [0, 1, 1, 1],
    [0, 0, 1, 1]
]
result = balancednessDetection(g)
print(f"Is the field balanced? {result}")


In [ ]:
def computeIPMatrix(field):
    field = np.array(field)
    w, h = field.shape

    # Create the new field with unique black cell identifiers
    newField = np.zeros_like(field, dtype=int)
    b = 1
    for i in range(w):
        for j in range(h):
            if field[i, j] == 1:
                newField[i, j] = b
                b += 1
    b -= 1  # since b was incremented one extra time

    rects = []
    for i in range(w):
        for j in range(h):
            if field[i, j] != 0:
                d = h
                for ii in range(i, w):
                    for jj in range(j, d):
                        if field[ii, jj] == 0:
                            d = jj
                            break
                        if i != ii or j != jj:
                            x = np.zeros(b, dtype=int)
                            x[newField[i:ii+1, j:jj+1][newField[i:ii+1, j:jj+1] > 0] - 1] = 1
                            rects.append(x)

    rects = np.array(rects).T if rects else np.empty((b, 0))
    if len(rects) > 0:
        rects = rects[~np.all(rects == 0, axis=1)]
        if len(rects) < b:
            rects = np.vstack((rects, np.zeros((b - len(rects), rects.shape[1]))))
        A = np.concatenate((rects, np.identity(b)), axis=1)
    else:
        A = np.identity(b)

    return A

field = [[1, 1, 0], [1, 1, 1], [0, 1, 1]]
field = np.reshape(field, (3, 3))

print(computeIPMatrix(field))


In [ ]:
def run(anz,size):
    def write_obj(vertices, output_obj_path='output.obj'):
        with open(output_obj_path, 'w') as obj_file:
            for vertex in vertices:
                obj_file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]}\n')

        print(f'OBJ file saved to {output_obj_path}')

    with open(f'fields{size}.obj', 'rb') as fileObj1:
        fields = pickle.load(fileObj1)

    matrices = [computeIPMatrix(f) for f in fields]
    for i in range(len(fields),anz):
        field = []
        matrix=[]
        while len(field) == 0:
            field = start(size, size)
            matrix = computeIPMatrix(field)
            
        saveMatrix(matrix,i)
        fields.append(field)
        matrices.append(matrix)

    with open('fieldsNew%s.obj'%size, 'wb') as handle:
        pickle.dump(fields, handle, protocol=pickle.HIGHEST_PROTOCOL)

#for i in range(5,25,5):
#    run(300, i)

In [ ]:
def run_walter_tu(range_to_use, timeOut):
    times = []
    balance_status = []

    for size in range_to_use:
        file_path = f"./matricesToTest/{size}.mat"
        file_path = os.path.expanduser(file_path)  # Expand the '~' to the full path
        
        start = subprocess.check_output(["date", "+%s%N"])
        start = int(re.sub(r'\D', '', start.decode("utf-8").strip()))  # Remove non-digit characters
        
        bash_command = ["/home/p/Desktop/cmr/build/cmr-tu", file_path]
        

        process = subprocess.Popen(bash_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        try:
            start_time = time.time()
            output, error = process.communicate(timeout=timeOut)  # Adding timeout
            end_time = time.time()
            combined_output = output.decode("utf-8") + error.decode("utf-8")
            
            end = subprocess.check_output(["date", "+%s%N"])
            end = int(re.sub(r'\D', '', end.decode("utf-8").strip()))  # Remove non-digit characters
            
            duration = end - start
            duration = end_time - start_time
            times.append(duration)    

            lines = combined_output.split('\n')
            #print(lines)
            for line in lines:
                if "Matrix IS" in line:
                    if "IS totally" in line:
                        balance_status.append(1)
                    elif "IS NOT totally" in line:
                        balance_status.append(0)
                        
        except subprocess.TimeoutExpired:
            process.kill()
            print(f"Process for instance {size} timed out.")
            times.append(float('inf'))  # Using 'inf' to indicate a timeout
            balance_status.append(-1)   # Using -1 to indicate an unknown status

    return times, balance_status

# Running the function and printing the results
times, tu_status = run_walter_tu(range(5),10)
print("Execution times (in nanoseconds):", times)
print("TUM (1) or Not TUM (0):", tu_status)
print("Length of tu_status:", len(tu_status))


In [ ]:
def run_walter_balanced(range_to_use, timeout_in_s):
    times = []
    balance_status = []
    execution_details = []

    for size in range_to_use:
        file_path = f"./matricesToTest/{size}.mat"
        file_path = os.path.expanduser(file_path)  # Expand the '~' to the full path
        
        start = subprocess.check_output(["date", "+%s%N"])
        start = int(re.sub(r'\D', '', start.decode("utf-8").strip()))  # Remove non-digit characters
        
        bash_command = ["/home/p/Desktop/cmr/build/cmr-balanced", file_path, "-s"]
        process = subprocess.Popen(bash_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        try:
            start_time = time.time()
            output, error = process.communicate(timeout=timeout_in_s)  # Adding timeout
            end_time = time.time()
            combined_output = output.decode("utf-8") + error.decode("utf-8")
            
            end = subprocess.check_output(["date", "+%s%N"])
            end = int(re.sub(r'\D', '', end.decode("utf-8").strip()))  # Remove non-digit characters
            
            duration = end_time - start_time
            times.append(duration)    

            combined_output = combined_output.lower()

            if "is balanced" in combined_output:
                balance_status.append(1)
            elif "is not balanced" in combined_output:
                balance_status.append(0)

            # Extracting timing information using regular expressions
            timing_info = re.findall(r'(\d+\.\d+)\sseconds', combined_output)
            timing_info = list(map(float, timing_info))
            execution_details.append(timing_info)

            #print("done with instance: " + str(size) +  " in " + str(duration) + " which is " + str(balance_status[-1]))

        except subprocess.TimeoutExpired:
            process.kill()
            print(f"Process for instance {size} timed out.")
            times.append(float('inf'))  # Using 'inf' to indicate a timeout
            balance_status.append(-1)   # Using -1 to indicate an unknown status
            execution_details.append([])

    return times, balance_status#, execution_details

# Running the function and printing the results
times, balance_status = run_walter_balanced(range(0, 2), 10)
print("Execution times (in seconds):", times)
print("Balanced (1) or Not Balanced (0):", balance_status)
print("Sum of balanced status:", sum(1 for status in balance_status if status == 1))


In [ ]:
from concurrent.futures import ThreadPoolExecutor
def saveMatrix(field, n):
    os.makedirs("matricesToTest", exist_ok=True)
    
    lengths1, lengths2 = field.shape
    header = f"{lengths1} {lengths2}"
    
    rows = [header] + [" ".join(map(str, map(int, row))) for row in field]

    file_path = f"matricesToTest/{n}.mat"
    with open(file_path, 'w') as f:
        f.write("\n".join(rows) + "\n")

def process_matrix(i, g):
    h = []
    while len(h) == 0:
        h = computeIPMatrix(g)
    saveMatrix(h, i)
    return i

def main(anz, x):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_matrix, i, x) for i in range(anz)]
        for future in futures:
            future.result()  # Wait for all futures to complete


In [ ]:
stepsize = range(5,26,5)
wdh = 1

numWalterTU = [[] for j in range(wdh)]
timeWalterTU = [[] for j in range(wdh)]
numWalterBAL = [[] for j in range(wdh)]
timeWalterBAL = [[] for j in range(wdh)]
numBal = [[] for j in range(wdh)]
timeUs = [[] for j in range(wdh)]

timeIP = [[] for j in range(wdh)]

for x in stepsize:
    print("SIZE", x, "x", x)
    t = time.time()

    with open(f'fields{x}.obj', 'rb') as fileObj1:
        fields = pickle.load(fileObj1)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_matrix, i, fields[i]) for i in range(len(fields))]
        for future in futures:
            future.result()  # Wait for all futures to complete

    print(f"Time taken for size {x}x{x}: {time.time() - t} seconds")


    for z in range(wdh):
        print("Iteration:", z)

        tt = time.time()
        balanced = []
        timeU = []
        for j in range(len(fields)): 
            t = time.time()
            balanced.append(balancednessDetection(fields[j]))
            timeU.append(time.time()-t)
        print(z,x, "Time of our algorithm" , time.time()-tt, "Number of balanced fields:", sum(balanced))
        numBal[z].append(balanced)
        timeUs[z].append(timeU)

        t, uni = run_walter_tu(range(len(fields)), 60)
        print(z,x, "Time of Matthias Walter TU: ",sum(t), "Number of unimodular fields:", sum(uni), "Number: ", len(uni), " problems: ", len([x for x in uni if x == -1]))
        numWalterTU[z].append(uni)
        timeWalterTU[z].append(t)


        t, bal = run_walter_balanced(range(len(fields)), 60)
        print(z,x, "Time of Matthias Walter Bal: ",sum(t), "Number of balanced fields:", sum(bal), "Number: ", len(bal), " problems: ", len([x for x in bal if x == -1]))
        numWalterBAL[z].append(bal)
        timeWalterBAL[z].append(t)

        t = time.time()
        #for j in range(len(fields)): 
        #     ip(fields[j])
        #timeIP[z].append(time.time()-t)
        #print(z,x, "Time IP: ",timeIP[z])

        print("\n")

    print("done with ", x, "round!\n\n")
    
print("our Algo")
#print(timeUs)
print("time walter Bal")
#print(timeWalterBAL)
print("time walter TU")
print(timeWalterTU)


In [ ]:
#np.save("timeWalterBAL",timeWalterBAL)
#np.save("numWalterBAL",numWalterBAL)
#np.save("timeWalterTU",timeWalterTU)

In [ ]:
sumW = [[0 for i in range(len(timeWalterBAL[0][0]))] for j in range(len(timeWalterBAL[0]))]
for i in range(len(timeWalterBAL)):
    for j in range(len(timeWalterBAL[i])):
        for jj in range(len(timeWalterBAL[i][j])):
            sumW[j][jj] = sumW[j][jj] + timeWalterBAL[i][j][jj]/wdh
print(len(sumW))
sumTU = [[0 for i in range(len(timeWalterTU[0][0]))] for j in range(len(timeWalterTU[0]))]
for i in range(len(timeWalterTU)):
    for j in range(len(timeWalterTU[i])):
        for jj in range(len(timeWalterTU[i][j])):
            sumTU[j][jj] = sumTU[j][jj] + timeWalterTU[i][j][jj]/wdh
print(len(sumTU))


sumU = [[0 for i in range(len(timeUs[0][0]))] for j in range(len(timeUs[0]))]
for i in range(len(timeUs)):
    for j in range(len(timeUs[i])):
        for jj in range(len(timeUs[i][j])):
            sumU[j][jj] = sumU[j][jj] + timeUs[i][j][jj]/wdh
print(len(sumU))

In [ ]:
results = ["TU-Test 5x5", "TU-Test 10x10", "TU-Test 15x15", "TU-Test 20x20", "TU-Test 25x25", "Balancedness 5x5", "Balancedness 10x10", "Balancedness 15x15", "Balancedness 20x20", "Balancedness 25x25", "Custom Balancedness 5x5", "Custom Balancedness 10x10", "Custom Balancedness 15x15", "Custom Balancedness 20x20", "Custom Balancedness 25x25"]
res = [*sumTU, *sumW, *sumU]
np.save("results2",res)
[sum(res[i]) for i in range(len(res))]

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "matplotlib"
x = np.arange(0.000001,10.01,0.000001)
res = np.load("results2.npy")
#res = [res[i] for i in range(0,5)]
anz = 100

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Define colors and linestyles based on index
colors_and_styles = [
    ('black', '--') if i < 5 else ('grey', ':') if i < 10 else ('lightgrey', '-')
    for i in range(len(res))
]

plt.figure(figsize=(10, 6))

# Iterate through each runtimes array
for i, (runtimes, (color, linestyle)) in enumerate(zip(res, colors_and_styles)):
    # Sort the runtimes
    sorted_runtimes = np.sort(runtimes)
    
    # Create an array representing the number of solved instances
    solved_instances = np.arange(1, len(runtimes) + 1)
    
    # Plot the number of solved instances over time
    plt.plot(sorted_runtimes, solved_instances, marker='o', markersize=3, linestyle=linestyle, color=color, label=f'Run {i+1}')
    
    # Annotate the first point with the run number
    #plt.text(sorted_runtimes[0], solved_instances[0], f'{i+1}', fontsize=5, ha='right', va='bottom')

# Add labels and title
plt.xlabel('Runtime (log scale)')
plt.ylabel('Number of Solved Instances')
plt.title('Number of Solved Instances Over Time for Multiple Runs')
plt.xscale('log')
plt.xlabel('Timeout (in seconds)')
plt.ylabel('Number of solved examples')
plt.ylim([0, 100])
plt.yticks([0,10,20,30,40,50,60,70,80,90,100])
plt.rcParams["figure.figsize"] = (5,4)

# Show the plot
plt.show()


plt.savefig('WaltervsBalanced.png',dpi=300, bbox_inches = "tight")
